In [1]:
from knowknow import *

In [2]:
from params import *

In [3]:
from load_db import db

loading variable anthro-wos-clean/_attributes from disk
loading variable anthro-wos-clean/groups from disk
loading variable anthro-wos-clean/group_reps from disk


In [4]:
cnt = db.by('c','fj').cits

loading variable anthro-wos-clean/ind ___ c.fj from disk


In [5]:
jj_counter = defaultdict(int)

for j1 in db.items('fj'):
    
    for (c2,j2),count2 in cnt.items():
        
        # undirected edges
        if j1 >= j2:
            continue
        
        if (c2,j1) not in cnt:
            continue
            
        count1 = cnt[(c2,j1)]
        cc = count1 * count2
        
        jj_counter[(j1,j2)] += cc

loading variable anthro-wos-clean/ind ___ fj from disk


In [6]:
# cosine similarity
jmag = {j: np.sqrt(
    np.sum([
        count**2
        for (_,fj),count in cnt.items() 
        if fj==j
    ])
) for j in db.items('fj')}

In [7]:
from csv import writer
Path('data').mkdir(exist_ok=True)
with open('data/jj_net.csv', 'w', encoding='utf8') as cf:
    w = writer(cf)
    w.writerow(['Source','Target','weight'])
    for (j1,j2),c in jj_counter.items():
        w.writerow([j1,j2,c / (jmag[j1]*jmag[j2])])

# yearly slices of journal structure?

In [8]:
jc = db.by('c','fj','fy').docs

loading variable anthro-wos-clean/doc ___ c.fj.fy from disk


In [9]:
jj_counter = defaultdict(int)

for j1 in db.items('fj'):
    
    for (c2,j2,y2),count2 in jc.items():
        
        # undirected edges
        if j1 >= j2:
            continue
        
        if (c2,j1,y2) not in jc:
            continue
            
        count1 = jc[(c2,j1,y2)]
        cc = count1 * count2
        
        jj_counter[(j1,j2,y2)] += cc

In [10]:
# individual files
for YY in range(1960,2010,5):
    
    yys = range(YY,YY+5)

    # cosine similarity
    jmag = {j: np.sqrt(
        np.sum([
            count**2
            for (_,fj,fy),count in jc.items() 
            if fj==j and fy in yys
        ])
    ) for j in db.items('fj')}

    from csv import writer
    
    with open('jj_net_%s.csv' % YY, 'w', encoding='utf8') as cf:
        w = writer(cf)
        w.writerow(['Source','Target','weight'])
        
        collapse = defaultdict(int)
        for (j1,j2,y), c in jj_counter.items():
            if y not in yys:
                continue
                
            collapse[(j1,j2)] += c
        
        for (j1,j2),c in collapse.items():
            w.writerow([j1,j2,c / (jmag[j1]*jmag[j2])])

In [11]:
from csv import writer        

with open('jj_net_TIME.csv', 'w', encoding='utf8') as cf:
    w = writer(cf)
    w.writerow(['Source','Target','weight','when_start','when_end'])

    # individual files
    for YY in range(1960,2020,5):

        yys = range(YY,YY+5)

        # cosine similarity
        jmag = {j: np.sqrt(
            np.sum([
                count**2
                for (_,fj,fy),count in jc.items() 
                if fj==j and fy in yys
            ])
        ) for j in db.items('fj')}

        collapse = defaultdict(int)
        for (j1,j2,y), c in jj_counter.items():
            if y not in yys:
                continue
                
            collapse[(j1,j2)] += c
        
        for (j1,j2),c in collapse.items():
            w.writerow([j1,j2,c / (jmag[j1]*jmag[j2]), YY, YY+5])

In [12]:
net = pd.read_csv('jj_net_TIME.csv')

## first determine overall arrangement

In [17]:
G = nx.Graph()

for i,r in net.iterrows():
    if r.weight < 0.05:
        continue
    G.add_edge(r.Source, r.Target, weight=r.weight)

In [19]:
from fa2 import ForceAtlas2

forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=True,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1.0,

                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=1.2,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=3.0,
                        strongGravityMode=False,
                        gravity=1.0,

                        # Log
                        verbose=True)

positions = forceatlas2.forceatlas2_networkx_layout(G, pos=None, iterations=2000)
nx.draw_networkx_nodes(G, positions, node_size=20, with_labels=False, node_color="blue", alpha=0.4)
nx.draw_networkx_edges(G, positions, edge_color="green", alpha=0.05)
plt.axis('off')
save_lim = (plt.xlim(), plt.ylim())
plt.show()

ModuleNotFoundError: No module named 'fa2'

In [ ]:
save_lim

## now plot subsets

In [15]:
smax_orig = None

for YY in range(1960,2020,5):

    G = nx.Graph()
    for i,r in net[net.when_start==YY].iterrows():
        if r.weight < 0.10:
            continue
            
        G.add_edge(r.Source, r.Target, weight=r.weight)

    zero_deg = [n for n,c in dict(nx.degree(G)).items() if c == 0]
    zero_deg = [x for x in zero_deg if x in positions]
    nonzero_deg = [n for n,c in dict(nx.degree(G)).items() if c != 0]
    #G.remove_nodes_from(zero_deg)
    
    plt.figure(figsize=(14,14))
    
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    
    sizes = {x: sum([db(fy=yyy, fj=x).cits for yyy in range(YY,YY+5)]) for x in zero_deg+nonzero_deg}
    
    if smax_orig is None:
        smax_orig = max(sizes.values())

    nx.draw_networkx_nodes(G, positions, nodelist=zero_deg, node_size=[sizes[x]*20/smax_orig for x in zero_deg], with_labels=False, node_color="black", alpha=1)
    nx.draw_networkx_nodes(G, positions, nodelist=nonzero_deg, node_size=[sizes[x]*20/smax_orig for x in nonzero_deg], with_labels=False, node_color="blue", alpha=0.8)
    
    nx.draw_networkx_edges(G, positions, edges=edges, edge_color=weights, alpha=0.5, edge_cmap=plt.cm.Blues)

    #ss = list(zip(*[(e,w) for e,w in nx.get_edge_attributes(G,'weight').items() if w>0.15]))
    #if len(ss):
    #    sedges,sweights = ss
    #    nx.draw_networkx_edges(G, positions, edgelist=sedges, edge_color=list(sweights), alpha=1)
        
    #top10 = Counter(dict(nx.degree(G, weight='weight'))).most_common(10)
    #top10 = [x[0] for x in top10]
    
    top10 = sorted(nonzero_deg, key=lambda x:-sizes[x])[:10]
    
    pos_higher = {}
    y_off = 15  # offset on the y axis
    for k, v in positions.items():
        pos_higher[k] = (v[0], v[1]+y_off)
    
    labels = {k:k for k in top10}
    #Now only add labels to the nodes you require (the hubs in my case)
    nx.draw_networkx_labels(G,pos_higher,labels,font_size=7,font_color='r')
    
        
    plt.title(YY)
    plt.axis('off')
    plt.xlim(*save_lim[0])
    plt.ylim(*save_lim[1])
    
    db.save_figure('net.%s.png' % YY)
    
    plt.show()

loading variable anthro-wos-clean/ind ___ fj.fy from disk


NameError: name 'positions' is not defined

<Figure size 1008x1008 with 0 Axes>

In [ ]:
nx.draw_networkx_nodes(G, positions, node_size=20, with_labels=False, node_color="blue", alpha=0.4)
nx.draw_networkx_edges(G, positions, edge_color="green", alpha=0.05)
plt.axis('off')
plt.show()